In [ ]:
# Scenario 31 - For Proposed Multimodal Fusion - Normailized Power Calculation - For Results!

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import scipy.io as scipyio
import matplotlib.pyplot as plt

from tqdm import tqdm

In [ ]:
# Absolute path of the folder containing the units' folders and scenarioX.csv
scenario_folder = r'C:\Users\Baqer\Desktop\V2X_CNN_All\Scenario31_64-Beams\scenario31'

In [ ]:
# Automatically fetch CSV (the only csv in folder)
try:
    csv_file = [f for f in os.listdir(scenario_folder) if f == 'best_epoch_Test.csv'][0]
    csv_path = os.path.join(scenario_folder, csv_file)
except IndexError:
    print("best_epoch_Test.csv not found in the scenario folder.")

dataframe = pd.read_csv(csv_path)
print(f'Columns: {dataframe.columns.values}')
print(f'Number of Rows: {dataframe.shape[0]}')

In [ ]:
N_BEAMS = 64
n_samples = 100
pwr_rel_paths = dataframe['original_unit1_pwr1'].values
pwrs_array = np.zeros((n_samples, N_BEAMS))

for sample_idx in tqdm(range(n_samples)):
    pwr_abs_path = os.path.join(scenario_folder, pwr_rel_paths[sample_idx])
    pwrs_array[sample_idx] = np.loadtxt(pwr_abs_path)

In [ ]:
selected_samples = [0, 2]
beam_idxs = np.arange(N_BEAMS) + 1
pwrs_array_selected = pwrs_array[selected_samples].T

for i, sample in enumerate(selected_samples):
    print(f"Sample {sample}:\n{pwrs_array_selected[:, i]}")

# Plotting the graph (optional)
plt.figure(figsize=(10, 6))
plt.plot(beam_idxs, pwrs_array_selected)
plt.legend([f'Sample {i}' for i in selected_samples])
plt.xlabel('Beam indices')
plt.ylabel('Power')
plt.grid()
plt.show()

In [ ]:
import pandas as pd

try:
    csv_file = [f for f in os.listdir(scenario_folder) if f == 'best_epoch_Test.csv'][0]
    csv_path = os.path.join(scenario_folder, csv_file)
except IndexError:
    print("best_epoch_Test.csv.csv not found in the scenario folder.")

dataframe = pd.read_csv(csv_path)

# Select the columns from top3_pred to top15_pred
selected_columns = dataframe.columns[dataframe.columns.get_loc('top3_pred'):dataframe.columns.get_loc('top15_pred')+1]

# Create a new DataFrame with the mentioned columns at the beginning
new_dataframe = pd.DataFrame()
new_dataframe['index'] = dataframe['index']
new_dataframe['link_status'] = dataframe['link_status']
new_dataframe['original_unit1_pwr1'] = dataframe['original_unit1_pwr1']
new_dataframe['top1_pred'] = dataframe['top1_pred']

# Split the remaining columns and add them to the new DataFrame
for i, column in enumerate(selected_columns[::-1], 1):
    new_columns = dataframe[column].astype(str).str.strip('[]').str.split(', ').apply(lambda x: pd.Series(x))
    new_columns.columns = [f'{column}_split_{j}' for j in range(len(new_columns.columns), 0, -1)]
    new_dataframe = pd.concat([new_columns, new_dataframe], axis=1)

# Reorder the columns
columns_order = ['index', 'link_status', 'original_unit1_pwr1', 'top1_pred'] + list(new_dataframe.columns[:-4])
new_dataframe = new_dataframe[columns_order]

# Save the updated DataFrame to a CSV file
new_dataframe.to_csv('splitted.csv', index=False)


In [ ]:
import os
import numpy as np
import pandas as pd

# Absolute path of the folder containing the units' folders and splitted.csv
scenario_folder = r'C:\Users\Baqer\Desktop\V2X_CNN_All\Scenario31_64-Beams\Power_Calculations-Proposed'
scenario_folder_main = r'C:\Users\Baqer\Desktop\V2X_CNN_All\Scenario31_64-Beams\scenario31'
# Read the 'splitted.csv' file into a DataFrame
csv_file = 'splitted.csv'
csv_path = os.path.join(scenario_folder, csv_file)
dataframe = pd.read_csv(csv_path)

# Columns to keep in the desired order
columns_order = ['index', 'original_unit1_pwr1', 'link_status', 'power_link_status', 'power_top1_pred'] + [col for col in dataframe.columns if col not in ['index', 'original_unit1_pwr1', 'link_status', 'power_link_status']]

N_BEAMS = 64
n_samples = dataframe.shape[0]
pwr_rel_paths = dataframe['original_unit1_pwr1'].values
pwrs_array = np.zeros((n_samples, N_BEAMS))

# Load the power data for each sample
for sample_idx in range(n_samples):
    pwr_abs_path = os.path.join(scenario_folder_main, pwr_rel_paths[sample_idx])
    pwrs_array[sample_idx] = np.loadtxt(pwr_abs_path)

# Iterate over the rows of the DataFrame and extract values
extracted_values = []
extracted_values_2 = []
for index, row in dataframe.iterrows():
    sample_idx = index  # Use the row index as the sample index
    list_index = row['link_status']  
    list_index_2 = row['top1_pred']
    
    # Adjust the list index by subtracting 1 to match the zero-based indexing in the power array
    adjusted_index = list_index - 1
    adjusted_index_2 = list_index_2 - 1
    
    # Check if the adjusted list index is within the range of the power array
    if 0 <= adjusted_index < pwrs_array.shape[1]:
        value = pwrs_array[sample_idx, adjusted_index]  # Get the corresponding element from the power array
        extracted_values.append(value)
    else:
        extracted_values.append(np.nan)
    if 0 <= adjusted_index_2 < pwrs_array.shape[1]:
        value_2 = pwrs_array[sample_idx, adjusted_index_2]
        extracted_values_2.append(value_2)
    else:
        extracted_values_2.append(np.nan)

# Add the extracted values as a new column named 'power_link_status' beside 'link_status'
dataframe.insert(dataframe.columns.get_loc('link_status') + 1, 'power_link_status', extracted_values)
dataframe.insert(dataframe.columns.get_loc('top1_pred') + 1, 'power_top1_pred', extracted_values_2)
# Reorder the columns according to the defined order
dataframe = dataframe[columns_order]

# Save the updated DataFrame to a new CSV file
output_csv_path = os.path.join(scenario_folder, 'power.rough.csv')
dataframe.to_csv(output_csv_path, index=False)


In [ ]:
import os
import numpy as np
import pandas as pd

# Absolute path of the folder containing the units' folders and splitted.csv
scenario_folder = r'C:\Users\Baqer\Desktop\V2X_CNN_All\Scenario31_64-Beams\Power_Calculations-Proposed'
scenario_folder_main = r'C:\Users\Baqer\Desktop\V2X_CNN_All\Scenario31_64-Beams\scenario31'
# Read the 'splitted.csv' file into a DataFrame
csv_file = 'splitted.csv'
csv_path = os.path.join(scenario_folder, csv_file)
dataframe = pd.read_csv(csv_path)

# Columns to keep in the desired order
columns_order = ['index', 'original_unit1_pwr1', 'link_status'] + \
    [col for col in dataframe.columns if col not in ['index', 'original_unit1_pwr1', 'link_status']]

N_BEAMS = 64
n_samples = dataframe.shape[0]
pwr_rel_paths = dataframe['original_unit1_pwr1'].values
pwrs_array = np.zeros((n_samples, N_BEAMS))

# Load the power data for each sample
for sample_idx in range(n_samples):
    pwr_abs_path = os.path.join(scenario_folder_main, pwr_rel_paths[sample_idx])
    pwrs_array[sample_idx] = np.loadtxt(pwr_abs_path)

# Create a new DataFrame to hold the power columns
power_dataframe = pd.DataFrame()

# Iterate over the rows of the DataFrame and extract values
for index, row in dataframe.iterrows():
    sample_idx = index  # Use the row index as the sample index
    list_index = row['link_status']
    
    # Adjust the list index by subtracting 1 to match the zero-based indexing in the power array
    adjusted_index = list_index - 1
    
    # Check if the adjusted list index is within the range of the power array
    if 0 <= adjusted_index < pwrs_array.shape[1]:
        value = pwrs_array[sample_idx, adjusted_index]  # Get the corresponding element from the power array
        power_dataframe.at[index, 'power_link_status'] = value

# Iterate over the split string columns and extract values
for column in dataframe.columns[dataframe.columns.str.contains('split')]:
    for index, row in dataframe.iterrows():
        sample_idx = index  # Use the row index as the sample index
        split_value = row[column]
        adjusted_index = split_value - 1
        
        # Check if the adjusted index is within the range of the power array
        if 0 <= adjusted_index < pwrs_array.shape[1]:
            value = pwrs_array[sample_idx, adjusted_index]  # Get the corresponding element from the power array
            power_column_name = f'power_{column}'
            power_dataframe.at[index, power_column_name] = value

# Save the power DataFrame to a new CSV file
output_csv_path = os.path.join(scenario_folder, 'power-rough2.csv')
power_dataframe.to_csv(output_csv_path, index=False)

# Save the updated DataFrame to a new CSV file without the power columns
dataframe_without_power = dataframe.drop(columns=[col for col in dataframe.columns if 'split' in col])
output_csv_path_without_power = os.path.join(scenario_folder, 'data_top-1_link_status.csv')
dataframe_without_power.to_csv(output_csv_path_without_power, index=False)

In [ ]:
import pandas as pd
import os
# Absolute path of the folder containing the power rough csv files
scenario_folder = r'C:\Users\Baqer\Desktop\V2X_CNN_All\Scenario31_64-Beams\Power_Calculations-Proposed'

# Read the 'power.rough.csv' and 'power-rough2.csv' files into DataFrames
power_csv_file = 'power.rough.csv'
power_csv_path = os.path.join(scenario_folder, power_csv_file)
power_dataframe = pd.read_csv(power_csv_path)

power9_csv_file = 'power-rough2.csv'
power9_csv_path = os.path.join(scenario_folder, power9_csv_file)
power9_dataframe = pd.read_csv(power9_csv_path)

# Merge the power and power9 DataFrames based on the index
merged_dataframe = pd.merge(power_dataframe, power9_dataframe, left_index=True, right_index=True)

# Remove the "power_link_status_y" column
merged_dataframe.drop(columns='power_link_status_y', inplace=True)

# Rename the "power_link_status_x" column to "power_link_status"
merged_dataframe.rename(columns={'power_link_status_x': 'power_link_status'}, inplace=True)

# Save the merged DataFrame to a new CSV file
output_csv_path = os.path.join(scenario_folder, '0final_results.csv')
merged_dataframe.to_csv(output_csv_path, index=False)
